BPE (Byte pair encoding) Guide:

1. Train:
    1. get text
    2. initialize vocab - map id: bytes. Start values are range 0-255: byte([0-255])
    3. initialize merges - map new_token_id: (prev_token_i, prev_token_j) - map of new token that formed from merge two existing tokens
    4. while vocab_size < target_vocab_size:
       - get frequencies of i, i+1 tokens. output: dict cat(token_i, token_j): freq
       - get token pair with max frequency
       - add to merges new_token_id: (token_i, token_j)
       - replace all max freq token pair with new_token_id
    5. for (token_i, token_j), new_token in merges:
       - add to vocab new_token = vocab\[token_i\] + vocab\[token_j\]

1. Inference:
    1. Encode:
       - encode text to byte tokens
       - while len(tokens) > 2 or no more merges performed: if pair tokens in merges: replace
    2. Decode: go through tokens and for each token get bytes from vocab: str(vocab\[token\])


In [16]:
self = BasicTokenizer()
vocab_size = 260
verbose = True
text = "hello world!"
tokens = list(text.encode('utf-8'))
vocab_size -= 256

merges = {}

progress_bar = tqdm(range(vocab_size)) if verbose else range(vocab_size)

for i in progress_bar:
    stats = self.get_freqs(tokens)
    max_freq_pair, _ = max(stats.items(), key=lambda x: x[1])
    new_token_id = 256 + i
    merges[new_token_id] = max_freq_pair
    tokens = self.replace_tokens_by_parent(tokens, max_freq_pair, new_token_id)

inv_merges = {v: k for k, v in merges.items()}
vocab = {i: bytes([i]) for i in range(256)}

for k, (p0, p1) in merges.items():
    vocab[k] = vocab[p0] + vocab[p1]

self.vocab, self.merges, self.inv_merges = vocab, merges, inv_merges

  0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [24]:
self.vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [23]:
vocab[104] + vocab[105]

b'hi'

In [21]:
merges

{256: (104, 101), 257: (256, 108), 258: (257, 108), 259: (258, 111)}

In [20]:
inv_merges

{(104, 101): 256, (256, 108): 257, (257, 108): 258, (258, 111): 259}

In [18]:
max_freq_pair

(258, 111)

I want achieve:

- vocabluary - the map of what pair of bytes were merged: text_corp -> f(x) -> utf8_bytes -> f(x) -> vocab
- decode - code, vocab -> f_decode(x, y) -> string
- encode - string -> f_encode(x) -> code, vocab

In [1]:
text = "“Why?’‘Because it is too small. “Five feet high is the door, and three abreast [first written four abreast] may enter it” say the runes. But Pryftan could not creep in a hole that size, not even when he was a young dragon, certainly not after he had devoured so many maidens of the valley.’‘It seems a pretty big hole,’ piped Bilbo. He loved maps, and in the hall there was a large one of the County Round (where he lived), with all his favourite walks marked on it in red ink. He was so interested he forgot to be shy and keep his mouth shut. ‘How could such an enormous door’ (he was a hobbit, remember) ‘be secret?’‘Lots of ways,’ said Bl[adorthin], ‘but which one of them we don’t know without looking. At the top of the other side of the page there is a list of the dwarves, which includes ‘Gandalf’; and against this my father afterwards wrote in pencil: ‘NB Gandalf was originally chief Dwarf (=Thorin) and Gandalf was called Bladorthin.’ The names of the dwarves in The Hobbit were taken from verses of a very ancient Norse poem called Völuspá, where many dwarf-names are given, and among them Gandalf. The only other difference in this original list is that Oi appears for Ori (in the Völuspá there is the name Ái). – Bladorthin became the name of a long-dead king who is mentioned once in The Hobbit (p. 230) but nowhere else. From what it says on the map I should say that there is a closed door which looks just like the side of the mountain – the ordinary dwarf’s way (I think I am right?)’‘Quite,’ said Gandalf. ‘But this rather alters things. There are fourteen of us – unless you are coming, Bladorthin. I had thought of going up along Running River from the Long Lake – if we can get so far! – and so to the Ruins of Dale Town. But we none of us liked the idea of the Front Gate. The River runs out of that great door, and out of it the Dragon comes too. Far too often.’‘That would have been no good,’ said Bl[adorthin], ‘without a mighty warrior; even a hero. I tried to find one, but I had to fall back (I beg your pardon, but I am sure you will understand – dragon-slaying is not I believe your speciality) – to fall back on little Bilbo [first written Mr Baggins].’‘The burglar,’ said Dwalin. ‘Precisely,’ said Blad[orthin], not allowing Bilbo time to object. ‘I told you last Thursday it would have to be a burglary not a battle, and a burglar I promised to find. I hope no one is going to say I put the sign on the wrong door again.’ He frowned so frightfully at Bilbo that the little man daren’t say anything though he was bursting with questions.‘Warriors are very busy fighting one another in far lands,’ went on Bl[adorthin], ‘and in this neighbour-hood there are none or few [struck out: left, of men, dwarves, elves or hobbits], not to speak of heroes. Swords in the world are mostly blunt, and axes used on trees and shields for dish-covers, and dragons comfortably far-off. But burglary is I think indicated in any case by the presence of the back door.’‘What is your plan?’ they all said. ‘To go to the back door, sit on the step – and think of one – if one does not”"

In [2]:
# text = "aaaBbaba"

In [3]:
tokens = list(text.encode('utf-8'))
len(tokens), tokens[:10]

(3212, [226, 128, 156, 87, 104, 121, 63, 226, 128, 153])

If I want to get vocab, I need to:
1. find two most frequent tokens
2. add them to map (a, b): new_token

In [4]:
def get_freqs(tokens):
    freq_stats = {}
    for i, j in zip(tokens, tokens[1:]):
        freq_stats[(i, j)] = freq_stats.get((i, j), 1) + 1
    return freq_stats

stats = get_freqs(tokens)
sorted(stats.items(), key=lambda x: x[1], reverse=True)[:5]

[((101, 32), 105),
 ((32, 116), 75),
 ((116, 104), 73),
 ((104, 101), 63),
 ((116, 32), 58)]

In [5]:
merges = {}
max_freq_pair, frq = max(stats.items(), key=lambda x: x[1])
merges[max_freq_pair] = 256
merges

{(101, 32): 256}

In [6]:
def replace_tokens_by_parent(tokens, target, parent):
    new_tokens = []

    i = 0
    while i < len(tokens):
        if i+1 < len(tokens) and (tokens[i], tokens[i+1]) == target:
            new_tokens.append(parent)
            i += 2
        else:
            new_tokens.append(tokens[i])
            i += 1
    
    return new_tokens

new_tokens = replace_tokens_by_parent(tokens, max_freq_pair, 256)

Fit vocab:
1. get tokens
2. get freqs of tokens
3. calculate max frequency pair
4. replace
5. repeat 2 step K times

In [7]:
N_MERGES = 42

tokens = list(text.encode('utf-8'))
merges = {}

for i in range(N_MERGES):
    stats = get_freqs(tokens)
    max_freq_pair, frq = max(stats.items(), key=lambda x: x[1])
    new_token_id = 256 + i
    merges[new_token_id] = max_freq_pair
    tokens = replace_tokens_by_parent(tokens, max_freq_pair, new_token_id)

In [8]:
len(list(text.encode('utf-8'))) / len(tokens)

1.5129533678756477

In [9]:
vocab = {i:bytes([i]) for i in range(256)}
inv_merges = {v: k for k, v in merges.items()}

for k, (p0, p1) in merges.items():
    vocab[k] = vocab[p0] + vocab[p1]

def decode(tokens):
    return b"".join(vocab.get(t) for t in tokens).decode("utf-8", errors="replace")

def encode(string):
    tokens = list(string.encode('utf-8'))
    while len(tokens) >= 2:
        new_tokens = []
        i = 0
        while i < len(tokens):
            if i+1 < len(tokens) and (tokens[i], tokens[i+1]) in inv_merges:
                new_tokens.append(inv_merges[tokens[i], tokens[i+1]])
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
    
        is_not_changed = len(tokens) == len(new_tokens)
        tokens = new_tokens
        if is_not_changed:
            break
    return tokens

decode(encode("HELLO WORLD!"))

'HELLO WORLD!'

In [10]:
decode(encode(text)) == text

True

# Exercise

https://www.youtube.com/watch?v=zduSFxRajkE&t=3417s&ab_channel=AndrejKarpathy

https://github.com/karpathy/minbpe/blob/master/exercise.md

In [4]:
def read_file(name):
    with open(name, "r") as file:
        return file.read()

text = read_file("taylorswift.txt")

In [5]:
from tqdm.auto import tqdm
from collections import Counter

class BasicTokenizer:
    def __init__(self):
        self.vocab, self.merges, self.inv_merges = {}, {}, {}
        pass

    def get_freqs(self, tokens):
        # Using Counter for efficient frequency calculation
        token_pairs = zip(tokens, tokens[1:])
        return Counter(token_pairs)

    def replace_tokens_by_parent(self, tokens, target, parent):
        # Improved token replacement for efficiency
        new_tokens = []
        i = 0
        while i < len(tokens):
            if i+1 < len(tokens) and (tokens[i], tokens[i+1]) == target:
                new_tokens.append(parent)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        return new_tokens
    
    def train(self, text, vocab_size, verbose=False):
        tokens = list(text.encode('utf-8'))
        vocab_size -= 256
        
        merges = {}
        
        progress_bar = tqdm(range(vocab_size)) if verbose else range(vocab_size)
        
        for i in progress_bar:
            stats = self.get_freqs(tokens)
            max_freq_pair, _ = max(stats.items(), key=lambda x: x[1])
            new_token_id = 256 + i
            merges[new_token_id] = max_freq_pair
            tokens = self.replace_tokens_by_parent(tokens, max_freq_pair, new_token_id)
        
        inv_merges = {v: k for k, v in merges.items()}
        vocab = {i: bytes([i]) for i in range(256)}
        
        for k, (p0, p1) in merges.items():
            vocab[k] = vocab[p0] + vocab[p1]
        
        self.vocab, self.merges, self.inv_merges = vocab, merges, inv_merges
        
    # def encode(self, text):
    #     # given a string text, return the token ids
    #     text_bytes = text.encode("utf-8") # raw bytes
    #     ids = list(text_bytes) # list of integers in range 0..255
    #     while len(ids) >= 2:
    #         stats = self.get_freqs(ids)
    #         pair = min(stats, key=lambda p: self.inv_merges.get(p, float("inf")))
    #         if pair not in self.inv_merges:
    #             break
    #         idx = self.inv_merges[pair]
    #         ids = self.replace_tokens_by_parent(ids, pair, idx)
    #     return ids

    def encode(self, text):
        tokens = list(text.encode('utf-8'))
        while len(tokens) >= 2:
            new_tokens = []
            i = 0
            while i < len(tokens):
                if i+1 < len(tokens) and (tokens[i], tokens[i+1]) in self.inv_merges:
                    new_tokens.append(self.inv_merges[tokens[i], tokens[i+1]])
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
        
            is_not_changed = len(tokens) == len(new_tokens)
            tokens = new_tokens
            if is_not_changed:
                break
        return tokens
        
    def decode(self, ids):
        return b"".join(self.vocab.get(t) for t in ids).decode('utf-8', errors="replace")


In [6]:
def read_file(name):
    with open(name, "r", encoding='utf-8') as file:
        return file.read()

# long_text = read_file('01 - The Fellowship Of The Ring.txt')
long_text = read_file('taylorswift.txt')

basic_tokenizer = BasicTokenizer()
basic_tokenizer.train(long_text, vocab_size=500, verbose=True)

basic_tokenizer.decode(basic_tokenizer.encode(long_text)) == long_text

  0%|          | 0/244 [00:00<?, ?it/s]

True

In [9]:
basic_tokenizer.vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [7]:
# !pip install seaborn
from IPython.display import display, HTML
import colorsys

def soft_pastel_colors(num_colors):
    # Generate colors in HSL space and convert to RGB, then to HEX for softer, muted tones
    colors = []
    for i in range(num_colors):
        # HSL: Hue, Saturation (lower for softness), Lightness (higher for pastel)
        hue = i / num_colors
        # saturation = 0.35  # Lower saturation for softness
        saturation = 0.80  # Lower saturation for softness
        lightness = 0.85  # Higher lightness for pastel
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        hex_color = "#{:02x}{:02x}{:02x}".format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))
        colors.append(hex_color)
    return colors

def get_color(token, color_list):
    return color_list[token % len(color_list)]

def print_colored_text(text, color_list):
    encoded_text = basic_tokenizer.encode(text)
    html_string = ''
    
    for token in encoded_text:
        color = get_color(token, color_list)
        decoded_char = basic_tokenizer.decode([token])  # Assuming this returns a string
        # print(f"{token} : {decoded_char}")
        html_string += f'<span style="background-color: {color};">{decoded_char}</span>'
    
    display(HTML(html_string))

# Example usage
# color_list = ['#FFDDDD', '#DDFFDD', '#DDDDFF', '#FFFFDD', '#DDFFFF', '#FFDDFF']
color_list =  soft_pastel_colors(100)

# This will display the colored text in a Jupyter Notebook cell
print_colored_text("Hello! How are you? February ", color_list)

In [8]:
basic_tokenizer.encode("Hello! How are you? February ")

[72, 358, 491, 33, 32, 72, 393, 32, 271, 256, 121, 321, 63, 32, 457]

# Make regex tokenizer

In [299]:
# import regex as re

# GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

# some_string = "Hello! How are you? February "
# splitted_text = re.findall(GPT4_SPLIT_PATTERN, some_string)

# tokens = [list(i.encode('utf-8')) for i in splitted_text]

# vocab_size = 300
# vocab_size -= 256

# merges = {}
# progress_bar = tqdm(range(vocab_size)) if True else range(vocab_size)

# for i in progress_bar:
#     stats = Counter()
    
#     for piece in tokens:
#         for pair in zip(piece[:-1], piece[1:]):
#             stats[pair] += 1
    
#     if not stats: break
    
#     max_pair = max(stats, key=lambda x: stats[x])
#     new_token_id = 256 + i
#     # new_token_id = len(stats)
#     merges[new_token_id] = max_pair

#     for j, piece in enumerate(tokens):
#         tokens[j] = basic_tokenizer.replace_tokens_by_parent(piece, max_pair, new_token_id)

# inv_merges = {v: k for k, v in merges.items()}
# vocab = {i: bytes([i]) for i in range(256)}

# for k, (p0, p1) in merges.items():
#     vocab[k] = vocab[p0] + vocab[p1]

  0%|          | 0/44 [00:00<?, ?it/s]

In [3]:
from IPython.display import display, HTML
import colorsys

def soft_pastel_colors(num_colors):
    colors = []
    for i in range(num_colors):
        # HSL: Hue, Saturation (lower for softness), Lightness (higher for pastel)
        hue = i / num_colors
        # saturation = 0.35  # Lower saturation for softness
        saturation = 0.80  # Lower saturation for softness
        lightness = 0.85  # Higher lightness for pastel
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        hex_color = "#{:02x}{:02x}{:02x}".format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))
        colors.append(hex_color)
    return colors

def get_color(token, color_list):
    return color_list[token % len(color_list)]

def print_colored_text(text, color_list, tokenizer):
    encoded_text = tokenizer.encode(text)
    html_string = ''
    
    for token in encoded_text:
        color = get_color(token, color_list)
        decoded_char = tokenizer.decode([token])  # Assuming this returns a string
        # print(f"{token} : {decoded_char}")
        html_string += f'<span style="background-color: {color};">{decoded_char}</span>'
    
    display(HTML(html_string))

In [2]:
from tqdm.auto import tqdm
from collections import Counter

import regex as re

class RegenxTokenizer(BasicTokenizer):
    def __init__(self):
        super().__init__()
        self.GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
        self.vocab, self.merges, self.inv_merges = {}, {}, {}

    def get_freqs(self, tokens):
        # Using Counter for efficient frequency calculation
        token_pairs = zip(tokens, tokens[1:])
        return Counter(token_pairs)

    def replace_tokens_by_parent(self, tokens, target, parent):
        # Improved token replacement for efficiency
        new_tokens = []
        i = 0
        while i < len(tokens):
            if i+1 < len(tokens) and (tokens[i], tokens[i+1]) == target:
                new_tokens.append(parent)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        return new_tokens
    
    def train(self, text, vocab_size, verbose=False):
        splitted_text = re.findall(self.GPT4_SPLIT_PATTERN, text)
        
        tokens = [list(i.encode('utf-8')) for i in splitted_text]
        
        vocab_size -= 256
        
        merges = {}
        progress_bar = tqdm(range(vocab_size)) if True else range(vocab_size)
        
        for i in progress_bar:
            stats = Counter()
            
            for piece in tokens:
                for pair in zip(piece[:-1], piece[1:]):
                    stats[pair] += 1
            
            if not stats: break
            
            max_pair = max(stats, key=lambda x: stats[x])
            new_token_id = 256 + i
            merges[new_token_id] = max_pair
        
            for j, piece in enumerate(tokens):
                tokens[j] = self.replace_tokens_by_parent(piece, max_pair, new_token_id)
        
        inv_merges = {v: k for k, v in merges.items()}
        vocab = {i: bytes([i]) for i in range(256)}
        
        for k, (p0, p1) in merges.items():
            vocab[k] = vocab[p0] + vocab[p1]
        
        self.vocab, self.merges, self.inv_merges = vocab, merges, inv_merges
        
    def encode_piece(self, text):
        tokens = list(text.encode('utf-8'))
        while len(tokens) >= 2:
            new_tokens = []
            i = 0
            while i < len(tokens):
                if i+1 < len(tokens) and (tokens[i], tokens[i+1]) in self.inv_merges:
                    new_tokens.append(self.inv_merges[tokens[i], tokens[i+1]])
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
        
            is_not_changed = len(tokens) == len(new_tokens)
            tokens = new_tokens
            if is_not_changed:
                break
        return tokens

    def encode_piece2(self, text):
        # given a string text, return the token ids
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255
        while len(ids) >= 2:
            stats = self.get_freqs(ids)
            pair = min(stats, key=lambda p: self.inv_merges.get(p, float("inf")))
            if pair not in self.inv_merges:
                break
            idx = self.inv_merges[pair]
            ids = self.replace_tokens_by_parent(ids, pair, idx)
        return ids

    def encode(self, text):
        splitted_text = re.findall(self.GPT4_SPLIT_PATTERN, text)
        return [item for i in splitted_text for item in self.encode_piece2(i)]
        
    def decode(self, ids):
        return b"".join(self.vocab.get(t) for t in ids).decode('utf-8', errors="replace")

def read_file(name):
    with open(name, "r", encoding='utf-8') as file:
        return file.read()

# long_text = read_file('taylorswift.txt')

long_text = """
Three Rings for the Elven-kings under the sky,
Seven for the Dwarf-lords in their halls of stone,
Nine for Mortal Men doomed to die,
One for the Dark Lord on his dark throne
In the Land of Mordor where the Shadows lie.
      One Ring to rule them all, One Ring to find them,
      One Ring to bring them all, and in the darkness bind them
In the Land of Mordor where the Shadows lie.

hello world!!!? (안녕하세요!) lol123 😉
"""

tokenizer = RegenxTokenizer()
tokenizer.train(long_text, vocab_size=1000, verbose=True)

tokenizer.decode(tokenizer.encode(long_text)) == long_text

NameError: name 'BasicTokenizer' is not defined

In [ ]:
tokenizer.vocab

In [85]:
color_list =  soft_pastel_colors(100)

test_text = """
Three Rings for the Elven-kings under the sky,
Seven for the Dwarf-lords in their halls of stone,
Nine for Mortal Men doomed to die,
One for the Dark Lord on his dark throne
In the Land of Mordor where the Shadows lie.
      One Ring to rule them all, One Ring to find them,
      One Ring to bring them all, and in the darkness bind them
In the Land of Mordor where the Shadows lie.

hello world!!!? (안녕하세요!) lol123 😉
"""

def print_colored_text(text, color_list, tokenizer):
    encoded_text = tokenizer.encode(text)
    html_string = '<div style="white-space: pre-wrap;">'
    
    for token in encoded_text:
        color = get_color(token, color_list)
        decoded_char = tokenizer.decode([token])  # Assuming this returns a string including spaces
        # Escaping HTML if necessary
        decoded_char = decoded_char.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        html_string += f'<span style="background-color: {color};">{decoded_char}</span>'
    
    html_string += '</div>'
    display(HTML(html_string))

    n_tokens = len(encoded_text)
    n_chars = len(text)

    print(f"Length of tokens: {n_tokens}, string length: {n_chars}, compression rate: {n_chars / n_tokens}")

print_colored_text(test_text, color_list, tokenizer)

Length of tokens: 98, string length: 420, compression rate: 4.285714285714286
